In [386]:
import os
import cv2
import random
import shutil
import scipy.io
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from scipy.linalg import inv, svd
from sklearn.neighbors import NearestNeighbors
from skfeature.function.similarity_based import fisher_score
from sklearn.preprocessing import LabelEncoder,OneHotEncoder

In [387]:
# Set dataset path (Google Drive or local folder)
d_p = Path("C:\\Users\\prade\\OneDrive\\Desktop\\PROJECT\\Datasets\\EXT_YALE_B")  # Change this based on your actual folder path
Total_image_per_dataset=60
Num_of_train_image=25
Num_of_test_image=Total_image_per_dataset-Num_of_train_image
Total_class=38

In [388]:
# === Create Directories if They Do Not Exist ===
P = []  # Training set
t = []  # Test set
shutil.rmtree("C:\\Users\\prade\\OneDrive\\Desktop\\PROJECT\\Datasets\\V2-G\\P", ignore_errors=True)
shutil.rmtree("C:\\Users\\prade\\OneDrive\\Desktop\\PROJECT\\Datasets\\V2-G\\t", ignore_errors=True)
train_labels = []
test_labels = []
train_path="C:\\Users\\prade\\OneDrive\\Desktop\\PROJECT\\Datasets\\V2-G\\P"
test_path="C:\\Users\\prade\\OneDrive\\Desktop\\PROJECT\\Datasets\\V2-G\\t"
os.makedirs(train_path, exist_ok=True)
os.makedirs(test_path, exist_ok=True)
IMAGE_SIZE = (32, 32)
,
# ==== Loop through each subfolder in order ====
subfolders = sorted(os.listdir(d_p))
processed_subfolders = 0
for folder in subfolders:
    person_path = os.path.join(d_p, folder)

    if os.path.isdir(person_path):
        # Filter only valid image files (to avoid hidden/system files)
        images = [f for f in os.listdir(person_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

        if len(images) >= Num_of_test_image:  
            test_images = random.sample(images, Num_of_test_image)
            train_images = [img for img in images if img not in test_images]
        else:
            # If fewer than 3 images, use all for training
            test_images = []
            train_images = images

        # ==== Processing Training Set ====
        if train_images:
            os.makedirs(os.path.join(train_path, folder), exist_ok=True)
            for img in train_images:
                img_path = os.path.join(person_path, img)
                #print("train", img_path)
                img_data = cv2.imread(img_path)
                if img_data is not None:
                    img_data = cv2.resize(img_data, IMAGE_SIZE)

                    # Flatten and normalize
                    img_vector = img_data.flatten().astype(np.float32)
                    norm = np.linalg.norm(img_vector, ord=2)
                    if norm != 0:
                        img_vector = img_vector / norm

                    # Append to training set
                    P.append(img_vector)
                    train_labels.append(folder)
                    # Copy image to train directory
                    shutil.copy(img_path, os.path.join(train_path, folder, img))

        # ==== Processing Test Set ====
        if test_images:
            os.makedirs(os.path.join(test_path, folder), exist_ok=True)
            for img in test_images:
                img_path = os.path.join(person_path, img)
                #print("test", img_path)
                img_data = cv2.imread(img_path)
                if img_data is not None:
                    img_data = cv2.resize(img_data, IMAGE_SIZE)

                    # Flatten and normalize
                    img_vector = img_data.flatten().astype(np.float32)
                    norm = np.linalg.norm(img_vector, ord=2)
                    if norm != 0:
                        img_vector = img_vector / norm

                    # Append to test set
                    t.append(img_vector)
                    test_labels.append(folder)
                    os.makedirs(os.path.join(person_path, folder), exist_ok=True)
                    # Copy image to test directory
                    shutil.copy(img_path, os.path.join(test_path, folder, img))

        processed_subfolders += 1

# Convert lists to numpy arrays
P = np.array(P)
t = np.array(t)

# Print shapes
print(f"Training set shape (P): {P.shape}")
print(f"Test set shape (t): {t.shape}")

# Convert labels to numpy arrays if needed
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

print(train_labels)
print(test_labels)
print(f"Training labels shape: {train_labels.shape}")
print(f"Test labels shape: {test_labels.shape}")

# Check number of processed subfolders
print(f"Number of processed subfolders: {processed_subfolders}")

Training set shape (P): (950, 3072)
Test set shape (t): (1330, 3072)
['01' '01' '01' '01' '01' '01' '01' '01' '01' '01' '01' '01' '01' '01'
 '01' '01' '01' '01' '01' '01' '01' '01' '01' '01' '01' '02' '02' '02'
 '02' '02' '02' '02' '02' '02' '02' '02' '02' '02' '02' '02' '02' '02'
 '02' '02' '02' '02' '02' '02' '02' '02' '03' '03' '03' '03' '03' '03'
 '03' '03' '03' '03' '03' '03' '03' '03' '03' '03' '03' '03' '03' '03'
 '03' '03' '03' '03' '03' '04' '04' '04' '04' '04' '04' '04' '04' '04'
 '04' '04' '04' '04' '04' '04' '04' '04' '04' '04' '04' '04' '04' '04'
 '04' '04' '05' '05' '05' '05' '05' '05' '05' '05' '05' '05' '05' '05'
 '05' '05' '05' '05' '05' '05' '05' '05' '05' '05' '05' '05' '05' '06'
 '06' '06' '06' '06' '06' '06' '06' '06' '06' '06' '06' '06' '06' '06'
 '06' '06' '06' '06' '06' '06' '06' '06' '06' '06' '07' '07' '07' '07'
 '07' '07' '07' '07' '07' '07' '07' '07' '07' '07' '07' '07' '07' '07'
 '07' '07' '07' '07' '07' '07' '07' '08' '08' '08' '08' '08' '08' '08'
 '08' '0

In [389]:
P.shape

(950, 3072)

In [390]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import os
from glob import glob
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# Load Pretrained ResNet50 Model (without classification head)
base_model = ResNet50(weights='imagenet', include_top=False, pooling='avg')
# Data Augmentation
def extract_features(img_path, model):
    """Extract deep features from an image using ResNet50"""
    img = image.load_img(img_path, target_size=(224, 224))  # Resize to model input size
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array = preprocess_input(img_array)  # Normalize
    features = model.predict(img_array)  # Extract features
    return features.flatten()  # Convert to 1D array
# Load dataset (Change path to your dataset)
dataset_path = "C:\\Users\\prade\\OneDrive\\Desktop\\PROJECT\\Datasets\\V2-G\\P"
classes = os.listdir(dataset_path)
X, y = [], []

for label, class_name in enumerate(classes):
    img_files = glob(os.path.join(dataset_path, class_name, "*.png"))  # Adjust for your format
    for img_path in img_files:
        features = extract_features(img_path, base_model)
        X.append(features)
        y.append(label)


# Convert to NumPy arrays
X = np.array(X)
y = np.array(y)
print(X.shape)
print(y.shape)
P=X
print(P.shape)

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 185ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 203ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 179ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 169ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 152ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 

In [391]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import os
from glob import glob

# Load Pretrained ResNet50 Model (without classification head)
base_model = ResNet50(weights='imagenet', include_top=False, pooling='avg')

def extract_features(img_path, model):
    """Extract deep features from an image using ResNet50"""
    img = image.load_img(img_path, target_size=(224, 224))  # Resize to model input size
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array = preprocess_input(img_array)  # Normalize
    features = model.predict(img_array)  # Extract features
    return features.flatten()  # Convert to 1D array

# Load dataset (Change path to your dataset)
dataset_path = "C:\\Users\\prade\\OneDrive\\Desktop\\PROJECT\\Datasets\\V2-G\\t"
classes = os.listdir(dataset_path)
I, a = [], []

for label, class_name in enumerate(classes):
    img_files = glob(os.path.join(dataset_path, class_name, "*.png"))  # Adjust for your format
    for img_path in img_files:
        features = extract_features(img_path, base_model)
        I.append(features)
        a.append(label)

# Convert to NumPy arrays
I = np.array(I)
a = np.array(a)
print(I.shape)
print(a.shape)
t=I
print(t.shape)

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 176ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 182ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 169ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 176ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 168ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/

In [392]:
#YET TO GENERALIZE
# Initialize a 750x750 matrix with zeros
matrix_size = Num_of_train_image*Total_class
block_size = Num_of_train_image
Y = np.zeros((matrix_size, matrix_size))

# Fill the matrix with the specified pattern
for i in range(0, matrix_size, block_size):
    Y[i:i + block_size, i:i + block_size] = 1

# Display the matrix
print(Y)
print("Y shape:",Y.shape)

[[1. 1. 1. ... 0. 0. 0.]
 [1. 1. 1. ... 0. 0. 0.]
 [1. 1. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 1. 1.]
 [0. 0. 0. ... 1. 1. 1.]
 [0. 0. 0. ... 1. 1. 1.]]
Y shape: (950, 950)


In [393]:
print(P.shape)
print(t.shape)

(950, 2048)
(1330, 2048)


FISHER SCORE IN ICSDLSR

In [395]:
# Fisher Score function for feature selection
#import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from skfeature.function.similarity_based import fisher_score

# Assume X has shape (600, 3072) and y has corresponding labels
R = P  # Replace with actual dataset
u = train_labels  # Binary classification labels
# Split data into training and testing sets

# Ensure correct shapes
print("X shape:", R.shape)  # Should be (n_samples, n_features)
print("y shape:", u.shape)  # Should be (n_samples,)

# Apply Fisher Score
score = fisher_score.fisher_score(R, u)
print("Fisher Score Computed Successfully")
selected_features = np.argsort(score)[-1000:]  # Select top 50 features

P1 = R[:, selected_features]
print(P1)

t1= t[:,selected_features]
print(P.shape)

P1 = P1.T
print(P1.shape)
print(t1.shape)

X shape: (950, 2048)
y shape: (950,)
Fisher Score Computed Successfully
[[0.         0.02158195 0.03928608 ... 0.01187269 0.01165274 0.00092303]
 [0.01768169 0.02874307 0.07645875 ... 0.02402238 0.01516443 0.07342871]
 [0.00318204 0.01397386 0.06595647 ... 0.00828493 0.04975586 0.08218054]
 ...
 [0.04890186 0.11578079 0.17240116 ... 0.01517668 0.02582759 0.24493688]
 [0.16084746 0.18570682 0.56457907 ... 0.04731442 0.01662921 0.35630628]
 [0.00972603 0.05290477 0.27287826 ... 0.02322642 0.04864168 0.36206856]]
(950, 2048)
(1000, 950)
(1330, 1000)


In [396]:
class ICS_DLSR:
    def __init__(self, P1, Y, lambda1, lambda2, lambda3, rho=1.01, mu_init=1e-8, mu_max=1e8):
        self.P1 = P1  # Data matrix (m x n)
        self.Y = Y  # Label matrix (c x n)

        # Dimensions
        m, n = P1.shape
        c = Y.shape[0]

        self.m = m
        self.n = n
        self.c = c

        # Regularization parameters
        self.lambda1 = lambda1
        self.lambda2 = lambda2
        self.lambda3 = lambda3

        # Penalty parameters
        self.rho = rho
        self.mu = mu_init
        self.mu_max = mu_max

        # Initialization
        self.Q = np.random.randn(self.c, self.m)  # Regression matrix (c x m)
        self.F = self.Q @ self.P1  # Auxiliary variable (c x n)
        self.C = np.zeros((self.c, self.n))  # Lagrange multiplier (c x n)
        self.E = np.zeros((self.c, self.n))  # Error matrix (c x n)

    def update_Q(self):
        G1 = self.Y + self.E
        G2 = self.F + (self.C / self.mu)

        A = (1 + self.mu) * (self.P1 @ self.P1.T) + self.lambda1 * np.eye(self.m)
        B = (G1 + self.mu * G2) @ self.P1.T

        self.Q = B @ np.linalg.inv(A)

    def update_F(self):
        H = self.Q @ self.P1 - (self.C / self.mu)
        self.F = np.zeros_like(H)

        for j in range(H.shape[0]):
            norm_Hj = np.linalg.norm(H[j, :], 2)
            if norm_Hj > (self.lambda2 / self.mu):
                self.F[j, :] = ((norm_Hj - (self.lambda2 / self.mu)) / norm_Hj) * H[j, :]
            else:
                self.F[j, :] = 0

    def update_E(self):
        U = self.Q @ self.P1 - self.Y
        self.E = np.zeros_like(U)

        for j in range(U.shape[0]):
            norm_Uj = np.linalg.norm(U[j, :], 2)
            if norm_Uj > self.lambda3:
                self.E[j, :] = ((norm_Uj - self.lambda3) / norm_Uj) * U[j, :]
            else:
                self.E[j, :] = 0

    def update_C_mu(self):
        self.C += self.mu * (self.F - self.Q @ self.P1)
        self.mu = min(self.rho * self.mu, self.mu_max)

    def compute_objective(self):
        term1 = 0.5 * np.linalg.norm(self.Y + self.E - self.Q @ self.P1, 'fro')**2
        term2 = (self.lambda1 / 2) * np.linalg.norm(self.Q, 'fro')**2
        term3 = self.lambda2 * np.sum([np.linalg.norm(self.F[i, :], 2) for i in range(self.c)])
        term4 = self.lambda3 * np.sum([np.linalg.norm(self.E[i, :], 2) for i in range(self.c)])
        term5 = (self.mu / 2) * np.linalg.norm(self.F - self.Q @ self.P1 + (self.C / self.mu), 'fro')**2
        return term1 + term2 + term3 + term4 + term5

    def fit(self, tol=1e-6, max_iter=500):
        for iteration in range(max_iter):
            Q_prev = self.Q.copy()

            # Update Q, F, E, and (C, µ)
            self.update_Q()
            self.update_F()
            self.update_E()
            self.update_C_mu()

            # Check convergence (if Q stops changing significantly)
            diff = np.linalg.norm(self.Q - Q_prev, 'fro')
            if diff < tol:
                print(f"Convergence reached at iteration {iteration + 1}.")
                break

    def get_results(self):
        return self.Q, self.E
    
# Regularization parameters
lambda1 = 0.01
lambda2 = 0.01
lambda3 = 0.01

# Create and fit the ICS_DLSR model
model = ICS_DLSR(P1, Y, lambda1, lambda2, lambda3)
model.fit()

# Retrieve the optimized matrices
Q1, E_opt = model.get_results()

print("Optimized Q:\n", Q1)
print(Q1.shape)
print("Optimized E:\n", E_opt)
print(E_opt.shape)
objective_value = model.compute_objective()
print("Computed Objective Value:", objective_value)

Optimized Q:
 [[-0.01490608  0.00978619  0.00060984 ... -0.03034977  0.02086238
  -0.01055895]
 [-0.01490608  0.00978619  0.00060984 ... -0.03034977  0.02086238
  -0.01055895]
 [-0.01490608  0.00978619  0.00060984 ... -0.03034977  0.02086238
  -0.01055895]
 ...
 [-0.03160828 -0.03026453  0.00775712 ... -0.04023735 -0.03791181
  -0.01042702]
 [-0.03160828 -0.03026453  0.00775712 ... -0.04023735 -0.03791181
  -0.01042702]
 [-0.03160828 -0.03026453  0.00775712 ... -0.04023735 -0.03791181
  -0.01042702]]
(950, 1000)
Optimized E:
 [[-0.04980307 -0.00636901 -0.0974081  ...  0.0117194  -0.07098413
   0.00053828]
 [-0.04980308 -0.00636902 -0.09740811 ...  0.0117194  -0.07098413
   0.00053827]
 [-0.04980307 -0.00636901 -0.0974081  ...  0.0117194  -0.07098413
   0.00053828]
 ...
 [-0.02472657  0.08294169  0.10606501 ... -0.19579012 -0.10146996
  -0.12074126]
 [-0.02472658  0.08294167  0.10606499 ... -0.19579013 -0.10146998
  -0.12074127]
 [-0.02472656  0.08294169  0.10606502 ... -0.19579011 -0.1

RFE IN ICSDLSR

In [398]:
# Define image size (resize all images to this size)
IMAGE_SIZE = (32,32)  # You can change this

# Get list of subfolders (each subfolder is a dog breed)
face = sorted([folder.name for folder in d_p.iterdir() if folder.is_dir()])
print("Face Found:", face)

# Initialize empty lists to store data

Z = []
# Loop through each breed (subfolder)
for f in face:
    f_path = d_p / f

    for img_file in f_path.glob("*.png"):  # Adjust file extensions if needed
        # Read and resize image
        img = cv2.imread(str(img_file))  # Read image
        img = cv2.resize(img, IMAGE_SIZE)  # Resize to fixed size
        img = img / 255.0  # Normalize pixel values (0 to 1)
        # Append corresponding label
        Z.append(f)
# Convert lists to NumPy arrays

print(P.shape)
Z = np.array(Z)
Z.shape
# Encode labels as numbers
label_encoder = LabelEncoder()
Z_encoded = label_encoder.fit_transform(Z)  # Converts breed names to integer labels
print(f"Target Matrix Z Shape: {Z_encoded.shape}")  # (n,)
print(f"Unique Labels: {len(np.unique(Z_encoded))}")

Face Found: ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39']
(950, 2048)
Target Matrix Z Shape: (2280,)
Unique Labels: 38


In [399]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectKBest, f_classif
# P: ResNet features from training images
# t: ResNet features from test images
# train_labels: labels for training set
# test_labels: labels for test set

selector = SelectKBest(score_func=f_classif, k=1000)
selector.fit(P, train_labels)

# Step 2: Select top features from both train and test sets
P2 = selector.transform(P)
t2 = selector.transform(t)

print("Selected Feature Mask:", selector.get_support())   # Boolean mask of selected features
print("P2 shape (train with selected features):", P2.shape)
print("t2 shape (test with selected features):", t2.shape)
# Optional: Transpose if your model expects features along rows
P2 = P2.T

Selected Feature Mask: [ True  True False ... False False False]
P2 shape (train with selected features): (950, 1000)
t2 shape (test with selected features): (1330, 1000)


In [400]:
class ICS_DLSR:
    def __init__(self, P2, Y, lambda1, lambda2, lambda3, rho=1.01, mu_init=1e-8, mu_max=1e8):
        self.P2 = P2  # Data matrix (m x n)
        self.Y = Y  # Label matrix (c x n)

        # Dimensions
        m, n = P2.shape
        c = Y.shape[0]

        self.m = m
        self.n = n
        self.c = c

        # Regularization parameters
        self.lambda1 = lambda1
        self.lambda2 = lambda2
        self.lambda3 = lambda3

        # Penalty parameters
        self.rho = rho
        self.mu = mu_init
        self.mu_max = mu_max

        # Initialization
        self.Q = np.random.randn(self.c, self.m)  # Regression matrix (c x m)
        self.F = self.Q @ self.P2  # Auxiliary variable (c x n)
        self.C = np.zeros((self.c, self.n))  # Lagrange multiplier (c x n)
        self.E = np.zeros((self.c, self.n))  # Error matrix (c x n)

    def update_Q(self):
        G1 = self.Y + self.E
        G2 = self.F + (self.C / self.mu)

        A = (1 + self.mu) * (self.P2 @ self.P2.T) + self.lambda1 * np.eye(self.m)
        B = (G1 + self.mu * G2) @ self.P2.T

        self.Q = B @ np.linalg.inv(A)

    def update_F(self):
        H = self.Q @ self.P2 - (self.C / self.mu)
        self.F = np.zeros_like(H)

        for j in range(H.shape[0]):
            norm_Hj = np.linalg.norm(H[j, :], 2)
            if norm_Hj > (self.lambda2 / self.mu):
                self.F[j, :] = ((norm_Hj - (self.lambda2 / self.mu)) / norm_Hj) * H[j, :]
            else:
                self.F[j, :] = 0

    def update_E(self):
        U = self.Q @ self.P2 - self.Y
        self.E = np.zeros_like(U)

        for j in range(U.shape[0]):
            norm_Uj = np.linalg.norm(U[j, :], 2)
            if norm_Uj > self.lambda3:
                self.E[j, :] = ((norm_Uj - self.lambda3) / norm_Uj) * U[j, :]
            else:
                self.E[j, :] = 0

    def update_C_mu(self):
        self.C += self.mu * (self.F - self.Q @ self.P2)
        self.mu = min(self.rho * self.mu, self.mu_max)

    def compute_objective(self):
        term1 = 0.5 * np.linalg.norm(self.Y + self.E - self.Q @ self.P2, 'fro')**2
        term2 = (self.lambda1 / 2) * np.linalg.norm(self.Q, 'fro')**2
        term3 = self.lambda2 * np.sum([np.linalg.norm(self.F[i, :], 2) for i in range(self.c)])
        term4 = self.lambda3 * np.sum([np.linalg.norm(self.E[i, :], 2) for i in range(self.c)])
        term5 = (self.mu / 2) * np.linalg.norm(self.F - self.Q @ self.P2 + (self.C / self.mu), 'fro')**2
        return term1 + term2 + term3 + term4 + term5

    def fit(self, tol=1e-6, max_iter=500):
        for iteration in range(max_iter):
            Q_prev = self.Q.copy()

            # Update Q, F, E, and (C, µ)
            self.update_Q()
            self.update_F()
            self.update_E()
            self.update_C_mu()

            # Check convergence (if Q stops changing significantly)
            diff = np.linalg.norm(self.Q - Q_prev, 'fro')
            if diff < tol:
                print(f"Convergence reached at iteration {iteration + 1}.")
                break

    def get_results(self):
        return self.Q, self.E
    
# Regularization parameters
lambda1 = 0.01
lambda2 = 0.01
lambda3 = 0.01

# Create and fit the ICS_DLSR model
model = ICS_DLSR(P2, Y, lambda1, lambda2, lambda3)
model.fit()

# Retrieve the optimized matrices
Q2, E_opt = model.get_results()

print("Optimized Q:\n", Q2)
print(Q1.shape)
print("Optimized E:\n", E_opt)
print(E_opt.shape)
objective_value = model.compute_objective()
print("Computed Objective Value:", objective_value)

Optimized Q:
 [[ 0.00984917 -0.03314255  0.00078671 ... -0.02050037 -0.02792962
   0.03600838]
 [ 0.00984917 -0.03314255  0.00078671 ... -0.02050037 -0.02792962
   0.03600838]
 [ 0.00984917 -0.03314255  0.00078671 ... -0.02050037 -0.02792962
   0.03600838]
 ...
 [-0.00312012  0.01104686  0.03075574 ...  0.01515398 -0.03108529
  -0.01507986]
 [-0.00312012  0.01104686  0.03075574 ...  0.01515398 -0.03108529
  -0.01507986]
 [-0.00312012  0.01104686  0.03075574 ...  0.01515398 -0.03108529
  -0.01507986]]
(950, 1000)
Optimized E:
 [[-0.06034888  0.0480514  -0.04846606 ... -0.01133949 -0.00724446
   0.01533912]
 [-0.06034888  0.04805141 -0.04846606 ... -0.01133949 -0.00724446
   0.01533912]
 [-0.06034888  0.04805141 -0.04846606 ... -0.01133949 -0.00724446
   0.01533912]
 ...
 [ 0.00714644  0.05723268  0.04032737 ... -0.07860699 -0.0488083
  -0.11513992]
 [ 0.00714644  0.05723269  0.04032737 ... -0.07860699 -0.0488083
  -0.11513992]
 [ 0.00714645  0.05723269  0.04032737 ... -0.07860699 -0.048

In [401]:
Qsum = np.vstack((Q1, Q2)) # Concatenate along columns
print("Concatenated Optimum Matrix Shape:", Qsum.shape)
print(Qsum)

Psum = sum((P1, P2)) # Concatenate along columns
print("Concatenated Optimum Matrix Shape:", Psum.shape)
print(Psum)
tsum = sum((t1, t2)) # Concatenate along columns
print("Concatenated Optimum Matrix Shape:", Psum.shape)
print(tsum)

Concatenated Optimum Matrix Shape: (1900, 1000)
[[-0.01490608  0.00978619  0.00060984 ... -0.03034977  0.02086238
  -0.01055895]
 [-0.01490608  0.00978619  0.00060984 ... -0.03034977  0.02086238
  -0.01055895]
 [-0.01490608  0.00978619  0.00060984 ... -0.03034977  0.02086238
  -0.01055895]
 ...
 [-0.00312012  0.01104686  0.03075574 ...  0.01515398 -0.03108529
  -0.01507986]
 [-0.00312012  0.01104686  0.03075574 ...  0.01515398 -0.03108529
  -0.01507986]
 [-0.00312012  0.01104686  0.03075574 ...  0.01515398 -0.03108529
  -0.01507986]]
Concatenated Optimum Matrix Shape: (1000, 950)
[[1.79926108e-03 8.04346055e-02 6.62432685e-02 ... 6.21801391e-02
  1.60847455e-01 3.46319452e-02]
 [1.16881676e-01 7.87562579e-02 2.95695662e-02 ... 3.87014955e-01
  3.32826465e-01 6.64943397e-01]
 [1.13185477e+00 1.21647525e+00 1.01945710e+00 ... 2.16574311e+00
  2.31589246e+00 1.38156426e+00]
 ...
 [1.18726883e-02 2.40223780e-02 8.28492921e-03 ... 7.87242949e-02
  4.73144166e-02 7.29407072e-02]
 [5.64023435

In [402]:
P_cap = Qsum @ Psum
t_cap = Qsum @ tsum.T
print(P_cap.shape)
print(t_cap.shape)

(1900, 950)
(1900, 1330)


CLASSIFICATION

In [404]:
from collections import Counter

k =  3
y_true = []
y_pred = []
# Initialize accuracy tracking
correct_predictions = 0
total_predictions = 0

# Loop through each test sample
for i in range(t_cap.shape[1]):  # 150 test samples
    test_image = t_cap[:, i]  # Get the ith test image

    # === Step 1: Compute Euclidean Distance ===
    distances = np.linalg.norm(P_cap - test_image[:, np.newaxis], axis=0)  # (600,)
    #print(distances)
    # === Step 2: Find k Nearest Neighbors ===
    nearest_indices = np.argsort(distances)[:k]  # Get indices of k nearest neighbors
    nearest_classes = train_labels[nearest_indices]  # Get the corresponding classes

    # === Step 3: Majority Voting ===
    predicted_class = Counter(nearest_classes).most_common(1)[0][0]
    expected_class = test_labels[i]
    y_true.append(expected_class)
    y_pred.append(predicted_class)
    # === Step 5: Accuracy Calculation ===
    if predicted_class == expected_class:
        correct_predictions += 1
    total_predictions += 1

# === Step 6: Overall Accuracy ===
accuracy = (correct_predictions / total_predictions) * 100
print(f"Overall Accuracy: {accuracy:.2f}%")
print(Num_of_train_image)

Overall Accuracy: 84.44%
25


In [405]:
import numpy as np
import seaborn as sns
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

# Example true labels and predicted labels


# Compute confusion matrix
cm = confusion_matrix(y_true, y_pred)
labels = ["Class 0", "Class 1"]  # Modify for multi-class problems
cm = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:\n", cm)

# Compute accuracy
accuracy = accuracy_score(y_true, y_pred)
print("Accuracy:", accuracy)

# Compute precision
precision = precision_score(y_true, y_pred, average='weighted')
print("Precision:", precision)

# Compute recall
recall = recall_score(y_true, y_pred, average='weighted')
print("Recall:", recall)

# Compute F1-score
f1 = f1_score(y_true, y_pred, average='weighted')
print("F1 Score:", f1)
# Plot confusion matrix

Confusion Matrix:
 [[29  0  1 ...  0  0  0]
 [ 0 28  0 ...  1  0  0]
 [ 1  0 25 ...  0  2  0]
 ...
 [ 0  0  0 ... 33  0  0]
 [ 0  0  1 ...  0 30  0]
 [ 1  0  0 ...  0  1 30]]
Accuracy: 0.8443609022556391
Precision: 0.8489765001377274
Recall: 0.8443609022556391
F1 Score: 0.8437752886052025
